In [7]:
import os, json
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
import openai
client = openai.OpenAI()

PROMPT = """
I have the following function

`get_weather`
`get_currecy`
`get_news`

All of them receive the name of a country as an arguemtn (i.e get_news('Spain))

Please anser with the name of the function that you would like me to run.

Please say nothing else, just the name of the functino with the arguemtns.

Answer th following question: What is the capital of Korea?

"""
response = client.chat.completions.create(
	model="gpt-4o-mini",
	messages=[{
		"role" : "user",
		"content" : PROMPT
	}]

)

response

ChatCompletion(id='chatcmpl-CETpQ7uLfEgtwxfUuKDjq7maKJKJF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="get_news('South Korea')", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1757567288, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_8bda4d3a2c', usage=CompletionUsage(completion_tokens=6, prompt_tokens=98, total_tokens=104, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [3]:
message =response.choices[0].message.content

message

"get_news('South Korea')"

In [4]:
import openai
client = openai.OpenAI()
messages = []

In [5]:
def get_weather(city):
	return "33 degrees celcius."

FUNCCTION_MAP = {
	'get_weather' : get_weather,
}

In [6]:
TOOLS = [
	{
		"type" : "function",
		"function" : {
			"name" : "get_weather",
			"description" : "A function to get the weather of a city.",
			"parameters" : {
				"type" : "object",
				"properties" : {
					"city" : {
						"type" : "string",
						"description" : "The name of the city to get the weather of."
					}
				},
				"required" : ["city"]
			}
		}
	}
]

In [ ]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
	if message.tool_calls > 0 :
		message.append({
			"role" : "assistant",
			"continue" : message.content or "",
			"tool_calls" : [{
				"id" : tools_call.id,
				"type" : "function",
				"functions" : {
					"name" : tool_call.function.name,
					"arguments" : tool_call.function.arguments
				}
			} for tool_calls in message.tool_calls]
		})

		for tool_call in message.tool_calls:
			function_name = tool_call.function.name
			arguments = tool_call.function.arguments

			print(f"Calling function: {function_name} with arguments {arguments}")

			try :
				arguments = json.loads(arguments)
			except json.JSONDecodeError:
				arguments = {}

			function_to_run = FUNCCTION_MAP.get(function_name)

			result = function_to_run(**arguments)

			message.append({
				"role" : "tool",
				"tool_call_id" : tool_call.id,
				"name" : function_name,
				"content" : result
			})
			
			

In [9]:
def call_ai():
		response = client.chat.completions.create(
			model="gpt-4o-mini",
			messages=messages,
			tools=TOOLS,

		)
		process_ai_response(response.choices[0].message)
		print(response)
		message = response.choices[0].message.content.tool_calls
		messages.append({
			"role" : "assistant",
			"content" : message
		})
		# return message
		print(f"AI: {message}")

In [10]:
while True:
	message = input("Send a message to LLM...")
	if message == 'quit':
		break
	else:
		messages.append({
			"role" : "user",
			"content" : message
		})
		print(f"User: {message}")
		call_ai()